# CropWise API - Google Colab Deployment

Notebook này triển khai API dự đoán bệnh cây ngô sử dụng ResNet18 trên Google Colab với GPU miễn phí.

## Hướng dẫn sử dụng:
1. Chạy các cells theo thứ tự từ trên xuống dưới
2. Setup ngrok token ở Cell 2
3. Upload model.pth ở Cell 4
4. Chạy Cell 9 để khởi động server


## Bước 1: Cài đặt Dependencies


In [ ]:
# Cài đặt PyTorch và torchvision với hỗ trợ CUDA (thay cu118 nếu Colab dùng CUDA khác)
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118

!pip install -q pyngrok

# Cài đặt các dependencies còn lại
!pip install -q fastapi==0.104.1 uvicorn==0.24.0 python-multipart==0.0.6 Pillow==10.1.0 numpy==1.24.3 pyngrok nest-asyncio

print("Đã cài đặt tất cả dependencies!")

/usr/lib/python3.12/pathlib.py:407: RuntimeWarning: coroutine 'Server.serve' was never awaited
  def _load_parts(self):


  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Đã cài đặt tất cả dependencies!


## Bước 2: Setup Ngrok (Chạy 1 lần)

1. Đăng ký tại https://dashboard.ngrok.com/signup
2. Lấy Authtoken từ dashboard
3. Thay `YOUR_NGROK_TOKEN_HERE` bằng token của bạn


In [ ]:
!ngrok config add-authtoken 35en9F2uu69hXmbUccRH2e3N9kh_6kwMzH7iDEMo8ttnrQXtK

print(" Đã setup ngrok!")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 Đã setup ngrok!


## Bước 3: Import Libraries


In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from PIL import Image
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
import torch.nn.functional as F
import io
import uvicorn
from pyngrok import ngrok
import nest_asyncio
from typing import List

nest_asyncio.apply()

print("Đã import tất cả libraries!")


Đã import tất cả libraries!


## Bước 4: Upload Model

**Chọn 1 trong 2 cách:**

### Cách 1: Từ Google Drive (Khuyến nghị)


In [ ]:
# CÁCH 1: Từ Google Drive
from google.colab import drive
drive.mount('/content/drive')

# THAY ĐỔI ĐƯỜNG DẪN NÀY theo vị trí file model.pth trong Google Drive của bạn
MODEL_PATH = '/content/drive/MyDrive/CropWise/model.pth'

print(f"Model path: {MODEL_PATH}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model path: /content/drive/MyDrive/CropWise/model.pth


### Cách 2: Upload trực tiếp (Uncomment nếu dùng cách này)


In [ ]:
# CÁCH 2: Upload trực tiếp (Uncomment nếu dùng cách này)
# from google.colab import files
# uploaded = files.upload()
# MODEL_PATH = 'model.pth'
# print(f"Model path: {MODEL_PATH}")


## Bước 5: Khởi tạo FastAPI App


In [ ]:
app = FastAPI(
    title="CropWise API",
    description="Corn Disease Detection API using ResNet18",
    version="1.0.0"
)

# CORS Middleware - Cho phép React Native gọi API
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Cho phép tất cả origins
    allow_credentials=True,
    allow_methods=["*"],  # Cho phép tất cả methods
    allow_headers=["*"],  # Cho phép tất cả headers
)

print("Đã khởi tạo FastAPI app!")


Đã khởi tạo FastAPI app!


## Bước 6: Load Model và Setup


In [ ]:
# Khởi tạo device (Colab có GPU miễn phí!)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang sử dụng thiết bị: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# Load model ResNet18
print("Đang load model ResNet18...")
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # 4 lớp bệnh

# Load trọng số đã train
print(f"Đang load weights từ: {MODEL_PATH}")
try:
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    model = model.to(device)
    model.eval()
    print("Đã load model thành công!")
except Exception as e:
    print(f"Lỗi khi load model: {e}")
    print("Vui lòng kiểm tra đường dẫn MODEL_PATH!")


Đang sử dụng thiết bị: cuda
GPU: Tesla T4
GPU Memory: 14.74 GB
Đang load model ResNet18...


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Đang load weights từ: /content/drive/MyDrive/CropWise/model.pth
Đã load model thành công!


## Bước 7: Định nghĩa Labels và Disease Info


In [ ]:
# Định nghĩa labels
labels = {
    0: 'Blight',
    1: 'Common_Rust',
    2: 'Gray_Leaf_Spot',
    3: 'Healthy'
}

labels_vi = {
    0: 'Bệnh Khô Lá',
    1: 'Bệnh Gỉ Sắt',
    2: 'Bệnh Đốm Lá Xám',
    3: 'Khỏe Mạnh'
}

# Mô tả bệnh
disease_info = {
    0: {
        'name': 'Bệnh Khô Lá (Blight)',
        'description': 'Bệnh do nấm gây ra, làm lá khô héo và chết dần.',
        'treatment': 'Sử dụng thuốc diệt nấm, cải thiện thoát nước, loại bỏ lá bệnh.'
    },
    1: {
        'name': 'Bệnh Gỉ Sắt (Common Rust)',
        'description': 'Bệnh nấm gây ra các đốm màu vàng cam trên lá.',
        'treatment': 'Phun thuốc diệt nấm chứa mancozeb hoặc chlorothalonil.'
    },
    2: {
        'name': 'Bệnh Đốm Lá Xám (Gray Leaf Spot)',
        'description': 'Bệnh nấm gây ra các vết đốm xám trên lá ngô.',
        'treatment': 'Luân canh cây trồng, sử dụng giống kháng bệnh, phun thuốc diệt nấm.'
    },
    3: {
        'name': 'Khỏe Mạnh (Healthy)',
        'description': 'Cây ngô hoàn toàn khỏe mạnh, không có dấu hiệu bệnh tật.',
        'treatment': 'Tiếp tục chăm sóc và theo dõi định kỳ.'
    }
}

# Transform ảnh
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Kích thước giống lúc train
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

print("Đã setup xong labels và transforms!")


Đã setup xong labels và transforms!


## Bước 8: Helper Function để Predict


In [ ]:
def predict_image(image: Image.Image):
    """
    Hàm helper để dự đoán từ một ảnh PIL
    """
    try:
        # Tiền xử lý ảnh
        input_tensor = transform(image).unsqueeze(0).to(device)

        # Dự đoán
        with torch.no_grad():
            output = model(input_tensor)
            probs = F.softmax(output, dim=1)
            pred_class = torch.argmax(probs, dim=1).item()
            confidence = probs[0][pred_class].item()

        # Tạo kết quả chi tiết
        all_predictions = {}
        for i in range(4):
            all_predictions[labels_vi[i]] = {
                "probability": float(probs[0][i] * 100),
                "label_en": labels[i]
            }

        return {
            "success": True,
            "predicted_class": labels[pred_class],
            "predicted_class_vi": labels_vi[pred_class],
            "confidence": float(confidence * 100),
            "disease_info": disease_info[pred_class],
            "all_predictions": all_predictions
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

print("Đã tạo helper function!")


Đã tạo helper function!


## Bước 9: Tạo API Endpoints


In [ ]:
@app.get("/")
async def root():
    """
    Health check endpoint
    """
    return {
        "message": "CropWise - Corn Disease Detection API",
        "status": "running",
        "model": "ResNet18",
        "device": str(device),
        "classes": labels_vi,
        "version": "1.0.0"
    }

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    """
    Endpoint để dự đoán bệnh từ 1 ảnh
    """
    try:
        # Đọc ảnh từ upload
        contents = await file.read()
        image = Image.open(io.BytesIO(contents)).convert("RGB")

        # Dự đoán
        result = predict_image(image)

        if result["success"]:
            print(f"Dự đoán: {result['predicted_class_vi']} ({result['confidence']:.2f}%)")
        else:
            print(f"Lỗi: {result.get('error', 'Unknown error')}")

        return result

    except Exception as e:
        print(f"Lỗi: {str(e)}")
        return {
            "success": False,
            "error": str(e)
        }

@app.post("/predict-batch")
async def predict_batch(files: List[UploadFile] = File(...)):
    """
    Endpoint để dự đoán bệnh từ nhiều ảnh (batch processing)
    """
    results = []
    processed = 0
    failed = 0

    try:
        print(f" Đang xử lý {len(files)} ảnh...")

        for idx, file in enumerate(files):
            try:
                # Đọc ảnh
                contents = await file.read()
                image = Image.open(io.BytesIO(contents)).convert("RGB")

                # Dự đoán
                result = predict_image(image)

                if result["success"]:
                    result["filename"] = file.filename or f"image_{idx}.jpg"
                    processed += 1
                    print(f"[{idx+1}/{len(files)}] {result['predicted_class_vi']} ({result['confidence']:.2f}%)")
                else:
                    result["filename"] = file.filename or f"image_{idx}.jpg"
                    result["error"] = result.get("error", "Unknown error")
                    failed += 1
                    print(f"[{idx+1}/{len(files)}] Lỗi: {result.get('error')}")

                results.append(result)

            except Exception as e:
                failed += 1
                results.append({
                    "filename": file.filename or f"image_{idx}.jpg",
                    "success": False,
                    "error": str(e)
                })
                print(f"[{idx+1}/{len(files)}] Exception: {str(e)}")

        return {
            "success": True,
            "processed": processed,
            "failed": failed,
            "results": results
        }

    except Exception as e:
        print(f"Lỗi batch processing: {str(e)}")
        return {
            "success": False,
            "processed": processed,
            "failed": failed,
            "results": results,
            "error": str(e)
        }

print("Đã tạo tất cả API endpoints!")
print("Endpoints:")
print("  - GET  / : Health check")
print("  - POST /predict : Single prediction")
print("  - POST /predict-batch : Batch prediction")


Đã tạo tất cả API endpoints!
Endpoints:
  - GET  / : Health check
  - POST /predict : Single prediction
  - POST /predict-batch : Batch prediction


In [ ]:
from pyngrok import ngrok
import uvicorn
import asyncio

print("Đang khởi động server...")

# Kết nối ngrok
public_url = ngrok.connect(8000)

print(f"\n{'='*60}")
print(f" SERVER ĐÃ SẴN SÀNG!")
print(f"{'='*60}")
print(f"Public URL: {public_url}")
print(f"API Endpoint: {public_url}/predict")
print(f"Batch Endpoint: {public_url}/predict-batch")
print(f"API Docs: {public_url}/docs")
print(f"Health Check: {public_url}/")
print(f"{'='*60}")
print(f"\n LƯU Ý: URL này sẽ thay đổi mỗi khi restart Colab!")
print(f"Copy URL này và cập nhật vào file: services/diseaseService.ts")
print(f"\nĐể giữ URL cố định, sử dụng ngrok domain (có phí)")
print(f"{'='*60}\n")

# Fix for asyncio.run() cannot be called from a running event loop in Colab
config = uvicorn.Config(app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)

# Run the server as an asyncio task
# This ensures it runs within the already existing event loop in Colab
asyncio.create_task(server.serve())

Đang khởi động server...

 SERVER ĐÃ SẴN SÀNG!
Public URL: NgrokTunnel: "https://indexless-lilyana-subprofessionally.ngrok-free.dev" -> "http://localhost:8000"
API Endpoint: NgrokTunnel: "https://indexless-lilyana-subprofessionally.ngrok-free.dev" -> "http://localhost:8000"/predict
Batch Endpoint: NgrokTunnel: "https://indexless-lilyana-subprofessionally.ngrok-free.dev" -> "http://localhost:8000"/predict-batch
API Docs: NgrokTunnel: "https://indexless-lilyana-subprofessionally.ngrok-free.dev" -> "http://localhost:8000"/docs
Health Check: NgrokTunnel: "https://indexless-lilyana-subprofessionally.ngrok-free.dev" -> "http://localhost:8000"/

 LƯU Ý: URL này sẽ thay đổi mỗi khi restart Colab!
Copy URL này và cập nhật vào file: services/diseaseService.ts

Để giữ URL cố định, sử dụng ngrok domain (có phí)



<Task pending name='Task-1' coro=<Server.serve() running at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69>>

## Hướng dẫn cập nhật React Native

Sau khi có URL từ ngrok, cập nhật file `services/diseaseService.ts`:

```typescript
const API_URL = 'https://xxxx-xxxx-xxxx.ngrok-free.app'; // URL từ ngrok
```

### Xử lý ngrok warning page:

Thêm header trong `diseaseService.ts`:

```typescript
headers: {
  'Accept': 'application/json',
  'ngrok-skip-browser-warning': 'true', // Bỏ qua warning
},
```


## Lưu ý quan trọng

1. **URL thay đổi:** Mỗi lần restart Colab, ngrok sẽ tạo URL mới
2. **Session timeout:** Colab free có giới hạn ~12 giờ, Pro ~24 giờ
3. **GPU:** Colab tự động cung cấp GPU T4 miễn phí
4. **Tốc độ:** Với GPU, dự đoán chỉ mất 0.1-0.5 giây (nhanh hơn Render 10-50 lần!)
